<a href="https://colab.research.google.com/github/eko-andri-prasetyo/Machine_Learning_Terapan-Predictive_Analytics/blob/main/soal_2_sistem_rekomendasi_ab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PERHATIAN!

Soal yang Anda kerjakan akan dinilai secara otomatis oleh sistem dari Dicoding. Oleh karena itu, Anda **tidak diperbolehkan mengubah nama fungsi yang sudah ditentukan** karena hal tersebut dapat memengaruhi proses penilaian.

Pastikan Anda membaca dan memperhatikan setiap instruksi dengan saksama serta menuliskan kode di antara tanda komentar yang telah disediakan.

> **# MULAI KODE DI SINI**


> **# AKHIRI KODE DI SINI**

---
Selain itu, Anda **tidak diperkenankan menggunakan *library* di luar yang telah ditentukan**. Seluruh soal telah dirancang agar dapat diselesaikan dengan *library* yang tersedia.

# A/B Testing

Anda telah mempelajari teori A/B testing pada modul khusus di kelas **Belajar Matematika untuk Data Science**. Pada bagian ini, Anda akan mengimplementasikan uji hipotesis melalui sebuah studi kasus A/B testing.

Namun, skenario yang digunakan berbeda dari contoh dalam modul tersebut.

---

Bayangkan Anda sedang mengembangkan sebuah website pribadi yang menyajikan berbagai artikel blog tentang teknologi terkini. Selain menyediakan konten bacaan, Anda juga menambahkan sistem rekomendasi artikel untuk menyarankan konten lain setelah pembaca menyelesaikan satu artikel.

Setelah beberapa waktu berjalan, Anda ingin mengetahui apakah penambahan fitur rekomendasi artikel di akhir halaman dapat mendorong pengguna untuk membaca lebih banyak artikel. Untuk menjawab pertanyaan tersebut, Anda memutuskan untuk menjalankan A/B testing selama 20 hari.

Metrik utama yang digunakan dalam pengujian ini adalah `articles_read`, yaitu jumlah artikel yang dibaca oleh pengguna dalam satu sesi. Anda akan membagi pengguna dalam dua kelompok pengujian.

* **Origin**: Kelompok yang tidak melihat fitur rekomendasi artikel (tampilan standar).
* **Varian**: Kelompok yang melihat tampilan dengan fitur rekomendasi artikel.

In [1]:
# TIDAK DIPERBOLEHKAN MENGGUNAKAN LIBRARY LAIN

import math, random, string
import numpy as np
import pandas as pd
import scipy.stats as stats

from dataclasses import dataclass

## Tugas 1: Menghasilkan Data Acak

Sebelum memulai proses pengujian, Anda perlu membuat data acak yang merepresentasikan pengguna yang mengunjungi website Anda.

**INGAT! TIDAK DIPERKENANKAN** untuk mengubah nama fungsi yang sudah didefinisikan.

## Membuat ID Acak

Kami telah menyiapkan fungsi khusus untuk menghasilkan ID acak, guna memastikan konsistensi output yang diharapkan.
Fungsi ini menghasilkan ID yang terdiri dari kombinasi huruf kapital. Sementara itu, nilai **ID** merupakan gabungan antara karakter [ASCII](https://www.ascii-code.com/articles/Beginners-Guide-to-ASCII) dan angka dengan panjang total **10 karakter**.

<font color="yellow"><strong>PERINGATAN: HARAP TIDAK MENGUBAH FUNGSI DI BAWAH INI!</strong></font>


In [2]:
# JANGAN MENGUBAH FUNGSI INI #

def create_unique_user_ids(num_users):
  """
  Menghasilkan sejumlah ID pengguna acak yang terdiri dari kombinasi huruf kapital ASCII dan angka
  dengan panjang tetap sebanyak 10 karakter.

  Parameter:
  - num_users (int): Jumlah ID unik yang ingin dihasilkan.

  Return:
  - user_ids (List[str]): List berisi ID unik hasil generate.
  """
  user_ids = []

  # Pengulangan hingga jumlah user_id yang dihasilkan sesuai dengan yang diminta
  while len(user_ids) < num_users:

      # Menghasilkan nilai random yang merupakan acak dari ascii dan string digits.
      new_id = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

      if new_id not in user_ids:
        user_ids.append(new_id)

  return list(user_ids)

### Tugas 1.1: Membuat Data Pengguna

Mari kita mulai dengan fungsi pertama. Fungsi ini akan menghasilkan data simulasi pengguna yang mengunjungi website Anda. Data tersebut terdiri dari tiga kolom utama berikut.

* **user\_id**: ID atau identitas unik dari masing-masing pengguna.
* **user\_type**: Menyatakan tipe pengguna, yaitu `origin` atau `varian`.
* **article\_read**: Menunjukkan jumlah artikel yang dibaca oleh pengguna.

Perlu diperhatikan bahwa nilai `article_read` bersifat diskret dan dapat dimodelkan menggunakan distribusi Poisson.

Untuk memastikan fungsi berjalan sesuai dengan yang diharapkan, pastikan Anda mengikuti setiap petunjuk yang telah diberikan.

<font color= 'yellow'><strong>CATATAN: JANGAN UBAH RANDOM SEED YANG SUDAH DITETAPKAN<strong></font>

In [3]:
def generate_df_ab_test(n_days=20):
  """
  Menghasilkan DataFrame yang menyimulasikan A/B testing untuk fitur sistem rekomendasi artikel.

  Parameters:
  - n_days (int): Jumlah hari yang disimulasikan (default: 20 hari)

  Returns:
  - df (pandas.DataFrame): DataFrame yang berisi hasil simulasi eksperimen A/B untuk kedua grup (origin dan varian).
  """

  ## JANGAN UBAH BAGIAN KODE DARI SINI
  random_state = np.random.default_rng(45)
  np.random.seed(45)

  daily_users = 104 # Banyaknya pengguna
  n_varian = int(daily_users * n_days * np.random.uniform(0.98, 1.02)) # Jumlah pengguna pada kelompok variation
  n_origin = int(n_varian * np.random.uniform(0.96, 0.98)) # Jumlah pengguna pada kelompok origin

  data_origin = stats.poisson.rvs(mu=4.7, size=n_origin, random_state=random_state)
  data_varian = stats.poisson.rvs(mu=5.3, size=n_varian, random_state=random_state)
  ## HINGGA BAGIAN KODE INI

  # MULAI KODE DI SINI

  user_ids = create_unique_user_ids(n_origin + n_varian)

  # Bagi ID untuk masing-masing kelompok
  origin_ids = user_ids[:n_origin]
  varian_ids = user_ids[n_origin:]

  # Buat DataFrame untuk kelompok origin
  origin_df = pd.DataFrame({
      "user_id": origin_ids,
      "user_type": "origin",
      "articles_read": data_origin
  })

  # Buat DataFrame untuk kelompok varian
  varian_df = pd.DataFrame({
      "user_id": varian_ids,
      "user_type": "varian",
      "articles_read": data_varian
  })


  # AKHIRI KODE DI SINI

  df = pd.concat([origin_df, varian_df]).sample(frac=1).reset_index(drop=True)
  return df

Data yang telah Anda hasilkan mencakup seluruh pengguna dari kedua kelompok. Untuk mendapatkan *insight* lebih mendalam, mari kita cari tahu rata-rata jumlah artikel yang dibaca oleh masing-masing kelompok.


In [4]:
# Pengujian: Silakan gunakan cell ini untuk menguji fungsi yang Anda buat. Pastikan output-nya sesuai dengan yang diharapkan.

df_articles = generate_df_ab_test(n_days=20)

origin_data = df_articles[df_articles['user_type'] == 'origin']['articles_read']
varian_data = df_articles[df_articles['user_type'] == 'varian']['articles_read']

print(f"{len(origin_data)} pengguna melihat website tanpa sistem rekomendasinya dengan rata-rata banyaknya artikel yang dibaca adalah {origin_data.mean():.2f}")
print(f"{len(varian_data)} pengguna melihat website dengan sistem rekomendasinya dengan rata-rata banyaknya artikel yang dibaca adalah {varian_data.mean():.2f}")

2058 pengguna melihat website tanpa sistem rekomendasinya dengan rata-rata banyaknya artikel yang dibaca adalah 4.72
2120 pengguna melihat website dengan sistem rekomendasinya dengan rata-rata banyaknya artikel yang dibaca adalah 5.38


Output yang diharapkan
```
2058 pengguna melihat website tanpa sistem rekomendasinya dengan rata-rata banyaknya artikel yang dibaca adalah 4.72
2120 pengguna melihat website dengan sistem rekomendasinya dengan rata-rata banyaknya artikel yang dibaca adalah 5.38
```

Sekilas, Anda mungkin menyadari bahwa pengguna yang mendapatkan fitur sistem rekomendasi cenderung memiliki rata-rata artikel yang dibaca lebih tinggi dibandingkan kelompok lainnya. Namun, sebagai data scientist yang bijak, Anda tidak bisa langsung menarik kesimpulan berdasarkan observasi awal semata.

Diperlukan penyelidikan lebih lanjut untuk memastikan bahwa perbedaan tersebut tidak terjadi secara kebetulan, tetapi benar-benar signifikan secara statistik.

Oleh karena itu, kita akan menggunakan **uji hipotesis** untuk mencari tahu jawabannya.

Perlu diperhatikan bahwa jumlah pengguna pada masing-masing kelompok tidak seimbang. Hal ini umum terjadi dalam praktik A/B testing, terutama ketika melibatkan proses pembagian acak.

---

Tugas Anda adalah melakukan uji hipotesis untuk memastikan adakah perbedaan *signifikan* antara rata-rata artikel yang dibaca oleh masing-masing kelompok. Karena kita membandingkan rata-rata, metode yang sesuai untuk kasus ini adalah **uji t dua sampel (t-test)** dengan **hipotesis nol** sebagai berikut.

> **Hipotesis nol (H₀):** Tidak ada perbedaan signifikan antara kedua kelompok.

Rumus t-statistik yang digunakan adalah berikut.

$t = \frac{(\bar{x}_{1} - \bar{x}_{2}) - (\mu_1 - \mu_2)}{\sqrt{\frac{s_{1}^2}{n_1} + \frac{s_{2}^2}{n_2}}}$

Artinya:
- $\bar{x}$ adalah rata-rata sampel,
- μ adalah rata-rata populasi,
- $s$ adalah standar deviasi, dan
- $n$ adalah ukuran sampel.

Uji ini dilakukan pada level pengguna (*user-level*) untuk memastikan terpenuhinya asumsi *independence*, karena setiap pengguna dianggap independen satu sama lain. Selain itu, ukuran sampel dalam data ini sudah mencukupi untuk menerapkan uji t dengan cukup andal.

## Menghitung t-statistik

Setelah data disiapkan, mari mulai dengan menghitung rumus statistik. Kita akan menyimpan setiap variabel rumus, seperti $\bar{x}$ dan lainnya dalam sebuah class.

Jangan khawatir jika Anda belum familier dengan fungsi ini. dataclass adalah decorator pada Python (dikenalkan sejak Python 3.7) yang secara otomatis menambahkan *boilerplate code* dalam sebuah class untuk menyimpan data.

Contohnya, Anda memiliki class berikut.

```
@dataclass
class my_course:
  course_id: int
  course_name: string

foo = my_course(course_name='Dicoding')
```

Anda dapat mengakses informasi `course_name` dari `foo` melalui sintaks `foo.course_name` yang akan mengembalikan string "Dicoding".


> Mengapa menggunakan dataclass? Anda perlu menyimpan parameter distribusi untuk setiap fitur. Misalnya, untuk weight dan height, diperlukan parameter μ dan σ. Jika dibuat terpisah, seperti miu_weight. sigma_weight, miu_height, sigma_height, akan tidak efektif karena terlalu panjang dan beragam.

__repr__ adalah method yang digunakan untuk menampilkan objek saat Anda print.

Tanpa `__repr__`:
```
<my_class object at 0x7f9f3b4d>
```

Dengan `__repr__`:
```
my_class(a='Dicoding')
```

In [5]:
@dataclass
class metrics_estimation:
  n: int
  xbar: float
  std: float

  def __repr__(self):
    return f"sample_params(n={self.n}, xbar={self.xbar:.3f}, std={self.std:.3f})"

### Tugas 1.2: Menghitung Metrik untuk t-statistik

Setelah setiap variabel disiapkan, silakan lengkapi fungsi di bawah ini yang bertujuan untuk menghitung nilai t-statistik berdasarkan rumus yang telah dijelaskan sebelumnya.

Dalam perhitungan ini, kita mengasumsikan bahwa data yang dimiliki berasal dari dua kelompok sampel dan kita ingin mengetahui apakah perbedaan antara kedua kelompok tersebut signifikan secara statistik.

Untuk masing-masing kelompok data, Anda hanya perlu menghitung tiga nilai utama berikut.

- ``n``: Ukuran sampel

- ``xbar``: Rata-rata dari sampel

- ``std``: Standar deviasi dari sampel


<details>
<summary>
<font color='yellow'>PETUNJUK!</font>
</summary>

- Fungsi len, np.mean, dan np.std akan berguna untuk Anda.
- Dalam konteks ini, kita akan menggunakan sampel, bukan populasi. Oleh karena itu pastikan saat menghitung standar deviasi, gunakan parameter `ddof=1` untuk menghitung ***sample standar deviation***.

In [6]:
def generate_metrics_for_t(data):
  """
  Menghitung metrik yang dibutuhkan untuk t-test berdasarkan data diskret.

  Parameters:
  - data (pandas.DataFrame): Data Sampel.

  Returns:
  - metrics: Metrik yang disimpan sebagai objek class `estimation_metrics`.
  """

  # MULAI KODE DI SINI

  n = len(data)

  # Hitung rata-rata sampel (x̄)
  xbar = np.mean(data)

  # Hitung standar deviasi sampel (s) dengan ddof=1
  std = np.std(data, ddof=1)

  # Simpan hasil perhitungan dalam objek dataclass metrics_estimation
  metrics = metrics_estimation(n=n, xbar=xbar, std=std)

  # AKHIRI KODE DI SINI

  return metrics

In [7]:
# Pengujian: Silakan gunakan cell ini untuk menguji fungsi yang Anda buat. Pastikan output-nya sesuai dengan yang diharapkan.

origin_metrics = generate_metrics_for_t(origin_data)
print(f"Kelompok origin: n = {origin_metrics.n}, rata-rata = {origin_metrics.xbar:.2f}, standar deviasi = {origin_metrics.std:.2f}")

varian_metrics = generate_metrics_for_t(varian_data)
print(f"Kelompok varian: n = {varian_metrics.n}, rata-rata = {varian_metrics.xbar:.2f}, standar deviasi = {varian_metrics.std:.2f}")


Kelompok origin: n = 2058, rata-rata = 4.72, standar deviasi = 2.16
Kelompok varian: n = 2120, rata-rata = 5.38, standar deviasi = 2.33


Output yang diharapkan
```
Kelompok origin: n = 2058, rata-rata = 4.72, standar deviasi = 2.16
Kelompok varian: n = 2120, rata-rata = 5.38, standar deviasi = 2.33
```

### Tugas 1.3: Menghitung Degree of Freedom

Anda telah berhasil membuat metrik yang dibutuhkan untuk menjalankan rumus *t-test*. Namun, selain metrik-metrik tersebut, Anda juga perlu menghitung satu nilai penting lainnya, yaitu **degree of freedom** (derajat kebebasan), yang memainkan peran kunci dalam *t-test*.

Degree of freedom (DoF) sangat penting karena menentukan **bentuk distribusi t** yang akan digunakan dalam menghitung nilai *p-value* dan *t-kritis*. Semakin besar nilai derajat kebebasan, distribusi t akan semakin mendekati distribusi normal.

Untuk menghitungnya, Anda dapat menggunakan rumus Satterthwaite berikut.

$$\text{Degrees of freedom } = \frac{\left[\frac{s_{1}^2}{n_1} + \frac{s_{2}^2}{n_2} \right]^2}{\frac{(s_{1}^2/n_1)^2}{n_1-1} + \frac{(s_{2}^2/n_2)^2}{n_2-1}}$$

Keterangan:

* $s_1, s_2$: standar deviasi dari masing-masing kelompok
* $n_1, n_2$: jumlah sampel dari masing-masing kelompok

<details>
<summary><font color='yellow'>PETUNJUK!</font></summary>

* Gunakan `np.square()` untuk menghitung kuadrat nilai.
* Dalam rumus di atas, **suffix 1** digunakan untuk menyatakan kelompok **origin**, dan **suffix 2** untuk kelompok **varian**.
* Untuk mengakses nilai dari objek `metrics` (dataclass), gunakan notasi titik/dot (`.`). Misalnya berikut.

  ```python
  origin_metrics.std  # Mengakses standar deviasi kelompok origin
  varian_metrics.n    # Mengakses jumlah sampel kelompok varian
  ```

</details>

In [8]:
def degrees_of_freedom(origin_metrics, varian_metrics):
  """
  Menghitung derajat kebebasan (degree of freedom) menggunakan rumus dari Satterthwaite.

  Parameters:
  - origin_metrics (metrics_estimation): Objek metrik yang berisi metrik untuk t-statistik dari kelompok origin.
  - varian_metrics (metrics_estimation): Objek metrik yang berisi metrik untuk t-statistik dari kelompok varian.

  Returns:
  - dof (numpy.float): Nilai derajat kebebasan.
  """

  # MULAI KODE DI SINI
  n1, s1 = origin_metrics.n, origin_metrics.std
  n2, s2 = varian_metrics.n, varian_metrics.std

  # Bagian pembilang rumus (numerator)
  numerator = np.square((s1**2 / n1) + (s2**2 / n2))

  # Bagian penyebut rumus (denominator)
  denominator = ((np.square(s1**2 / n1)) / (n1 - 1)) + ((np.square(s2**2 / n2)) / (n2 - 1))

  # Hasil derajat kebebasan
  dof = numerator / denominator

  # AKHIRI KODE DI SINI

  return dof

In [9]:
# Pengujian: Silakan gunakan cell ini untuk menguji fungsi yang Anda buat. Pastikan output-nya sesuai dengan yang diharapkan.

# Derajat kebebasan untuk sampel
dof_ab = degrees_of_freedom(origin_metrics, varian_metrics)
print(f"Derajat kebebasan (DoF) untuk data A/B Test: {dof_ab:.2f}")

Derajat kebebasan (DoF) untuk data A/B Test: 4168.28


Output yang diharapkan
```
Derajat kebebasan (DoF) untuk data A/B Test: 4168.28
```

### Tugas 1.4: Menghitung t-statistic

Pada tahap ini, Anda telah memiliki semua informasi yang dibutuhkan untuk menghitung nilai *t-statistic* dalam uji hipotesis dua sampel independen. Nilai *t-statistic* dihitung menggunakan rumus berikut.

$$
t = \frac{(\bar{x}_{1} - \bar{x}_{2}) - (\mu_1 - \mu_2)}{\sqrt{\frac{s_{1}^2}{n_1} + \frac{s_{2}^2}{n_2}}}
$$

Keterangan:

* \$\bar{x}\_1, \bar{x}\_2\$: rata-rata dari masing-masing sampel.
* \$s\_1, s\_2\$: standar deviasi dari masing-masing sampel.
* \$n\_1, n\_2\$: jumlah sampel pada masing-masing kelompok.
* \$(\mu\_1 - \mu\_2)\$: perbedaan rata-rata populasi yang **diasumsikan** dalam *null hypothesis*.


<details>
<summary><font color='yellow'>PETUNJUK!</font></summary>

* Berdasarkan *null hypothesis*, kita **mengasumsikan tidak ada perbedaan** antara dua populasi. Oleh karena itu, nilai \$(\mu\_1 - \mu\_2)\$ biasanya dianggap **0**.
* Anda dapat menggunakan `np.sqrt(...)` untuk menghitung akar kuadrat pada bagian penyebut (denominator) dari rumus.

In [10]:
def t_statistics_form(origin_metrics, varian_metrics):
    """
    Menghitung nilai statistik t dari dua sampel independen (misalnya, kelompok kontrol dan kelompok variasi)
    berdasarkan asumsi bahwa varians antar kelompok tidak sama (Welch's t-test).

    Parameters:
    - origin_metrics: Objek metrik dari kelompok origin, yang memuat ukuran sampel (n), rata-rata (x̄), dan standar deviasi (std).
    - varian_metrics: Objek metrik dari kelompok varian, yang juga memuat nilai n, x̄, dan std.

    Returns:
    - t (float): Nilai t-statistic yang dihitung berdasarkan rumus dua sampel independen.
    """

    # MULAI KODE DI SINI

    n1, xbar1, s1 = origin_metrics.n, origin_metrics.xbar, origin_metrics.std
    n2, xbar2, s2 = varian_metrics.n, varian_metrics.xbar, varian_metrics.std

    # Asumsi null hypothesis: (μ1 - μ2) = 0
    numerator = (xbar1 - xbar2) - 0
    denominator = np.sqrt((s1**2 / n1) + (s2**2 / n2))

    t = numerator / denominator

    # AKHIRI KODE DI SINI

    return t


In [11]:
# Pengujian: Silakan gunakan cell ini untuk menguji fungsi yang Anda buat. Pastikan output-nya sesuai dengan yang diharapkan.

t = t_statistics_form(origin_metrics, varian_metrics)
print(f"t statistic untuk AB test: {t:.2f}")

t statistic untuk AB test: -9.48


Output yang diharapkan
```
-9.48
```

## Menolak atau Menerima Hipotesis

Setelah melakukan semua perhitungan dengan rumus *t-statistic*, sekarang saatnya menentukan apakah hipotesis nol (*null hypothesis*) sebaiknya **ditolak** atau **tidak**.

Lengkapilah fungsi di bawah ini yang bertujuan untuk mengambil keputusan tersebut menggunakan **metode p-value** berdasarkan hal berikut.

* Nilai *t-statistic*.
* Derajat kebebasan (*Degrees of Freedom*).
* Tingkat signifikansi (*alpha*).

Fungsi ini menerapkan **uji dua sisi (two-sided test)** karena kita ingin mengetahui apakah terdapat perbedaan yang signifikan, baik itu **lebih tinggi** maupun **lebih rendah** dari nilai ekspektasi.

---

### Pemahaman P-Value dalam Uji Dua Sisi

Dalam konteks ini, *p-value* merepresentasikan **probabilitas** untuk memperoleh nilai *t-statistic* yang **sama ekstremnya atau lebih ekstrem** dibanding nilai yang diamati, **jika hipotesis nol benar**.

Anda dapat menggunakan fungsi **CDF (Cumulative Distribution Function)** dari distribusi *t* untuk menghitung probabilitas bahwa nilai yang diperoleh **kurang dari atau sama dengan** nilai yang diberikan.

Oleh karena itu, p-value dapat dihitung dengan rumus berikut.

$$
p = 2 \times (1 - \text{CDF}(|t|))
$$

Penjelasan:

* Gunakan **nilai absolut** dari *t-statistic* karena uji dua sisi memperhitungkan kedua ekor distribusi (baik negatif maupun positif).
* Kalikan hasilnya dengan **2** untuk mencerminkan kedua sisi distribusi.
* Bandingkan hasil p-value tersebut dengan nilai **α (alpha)**.

  * Jika **p-value < alpha**, hipotesis nol **ditolak**.
  * Jika **p-value ≥ alpha**, hipotesis nol **tidak ditolak**.

---

<details>
<summary><font color='yellow'>PETUNJUK</font></summary>

* Gunakan fungsi `cdf` dari modul `scipy.stats.t` untuk menghitung probabilitas kumulatif dari distribusi *t*.
* Masukkan `abs(t_statistic)` sebagai parameter dalam fungsi `cdf` karena ini adalah uji dua sisi.
* Kalikan hasil `1 - CDF(...)` dengan 2.
* Jika hasil akhir p-value lebih kecil dari `alpha`, **hipotesis nol ditolak**.

</details>

In [12]:
def reject_nh_t_statistic(t_statistic, dof, alpha=0.05):
    """
    Menentukan apakah hipotesis nol dalam uji t (t-test) harus ditolak atau tidak,
    berdasarkan nilai t-statistik, derajat kebebasan (degrees of freedom), dan tingkat signifikansi (alpha).

    Parameter:
    - t_statistic (float): Nilai t-statistik yang sudah dihitung dari dua sampel.
    - dof (float): Derajat kebebasan hasil perhitungan dari dua sampel.
    - alpha (float, opsional): Tingkat signifikansi yang digunakan. Nilai default adalah 0.05.

    Return:
    - reject (bool): Bernilai True jika hipotesis nol harus ditolak, dan False jika tidak ditolak.
    """

    reject = False

    # MULAI KODE DI SINI
    # Langkah 1: Hitung nilai p-value untuk uji dua sisi (two-sided test)
    # Gunakan nilai absolut dari t_statistic dan fungsi distribusi kumulatif (CDF) dari distribusi t
    # Jangan lupa kalikan hasilnya dengan 2 karena ini adalah uji dua sisi
    p_value = 2 * (1 - stats.t.cdf(abs(t_statistic), df=dof))

    # Bandingkan p-value dengan alpha
    if p_value < alpha:
        reject = True

    # AKHIRI KODE DI SINI

    return reject


In [13]:
# Pengujian: Silakan gunakan cell ini untuk menguji fungsi yang Anda buat. Pastikan output-nya sesuai dengan yang diharapkan.

alpha = 0.05
tolak_hipotesis_nol = reject_nh_t_statistic(t, dof_ab, alpha)

print(f"Hipotesis nol {'' if tolak_hipotesis_nol else 'tidak '}dapat ditolak pada tingkat signifikansi {alpha:.2f}.\n")

pesan = "" if tolak_hipotesis_nol else "Tidak "
print(f"{pesan}Ada cukup bukti statistik untuk menolak H0.\n"
      f"Dengan kata lain, dapat disimpulkan bahwa ada perbedaan yang {pesan}signifikan secara statistik antara rata-rata kedua kelompok.")

Hipotesis nol dapat ditolak pada tingkat signifikansi 0.05.

Ada cukup bukti statistik untuk menolak H0.
Dengan kata lain, dapat disimpulkan bahwa ada perbedaan yang signifikan secara statistik antara rata-rata kedua kelompok.


Output yang diharapkan
```
Hipotesis nol dapat ditolak pada tingkat signifikansi 0.05.

Ada cukup bukti statistik untuk menolak H0.
Dengan kata lain, dapat disimpulkan bahwa ada perbedaan yang signifikan secara statistik antara rata-rata kedua kelompok.
```

Selamat! Anda berhasil menjalankan uji hipotesis menggunakan two-tailed t-test berdasarkan data sintetis.

Berdasarkan hasil yang didapat, ada perbedaan yang signifikan secara statistik antara rata-rata kedua kelompok A/B test. Jadi, pengetahuan yang Anda dapat sebelumnya, yakni orang yang mendapatkan sistem rekomendasi menjadi lebih sering membaca artikel, tidak terjadi secara kebetulan.